In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import plotly.express as px

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
data_desc = pd.read_csv('data/data_description.csv', encoding='latin-1')
df = pd.read_csv('data/investigation_train_large_checked.csv')
df_synth = pd.read_csv('data/synth_data_for_training.csv')

In [ ]:
figures = []
for column in df.columns:
    fig = px.histogram(df, x=column, title=f"Distribution of {column}", nbins=30)
    fig.show()

In [ ]:
# Compute the correlation matrix
corr_matrix = df.corr()

# Plot the correlation matrix
plt.figure(figsize=(22, 18))
sns.heatmap(corr_matrix, annot=False, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Feature Correlation Matrix")
plt.savefig("correlation_matrix.png", dpi=300)
plt.show()

# Identify highly correlated features (threshold: 0.9)
threshold = 0.9
high_corr_pairs = np.where(np.abs(corr_matrix) > threshold)
high_corr_pairs = [
    (df.columns[i], df.columns[j]) 
    for i, j in zip(*high_corr_pairs) if i != j and i < j
]

# Display highly correlated feature pairs
if high_corr_pairs:
    print("Highly correlated feature pairs (threshold > 0.9):")
    for pair in high_corr_pairs:
        print(pair)
else:
    print("No highly correlated feature pairs found.")
